In [1]:
import pandas as pd
from functools import partial
import typing
import xml.etree.ElementTree as ET

In [2]:
def extract_authors(filepath: str) -> pd.DataFrame:
    """
    Extracts author information from a csv.
    
    ARGS:
        filepath (str) | path to csv
        
    RETURNS:
        (pd.DataFrame) | DataFrame of author info
    """
    
    authors = pd.read_csv(filepath,names=['author','affiliation'])
    
    return authors

In [3]:
# Build xml
def build_creators_xml(authors: pd.DataFrame) -> str:
    
    creators = ET.Element('creators')
    
    gen = partial(generate_creator,parent=creators)
    
    _authors = authors.apply(gen,axis=1)
    
    tree = ET.ElementTree(creators)
    
    return tree

In [4]:
def generate_creator(data: pd.Series, parent) -> ET.Element:
        
    names = extract_name(str(data[0]))
        
    creator = ET.SubElement(parent,'creator')
    creator_name = ET.SubElement(creator,'creatorName',nameType = "Personal")
    given_name = ET.SubElement(creator,'givenName')
    family_name = ET.SubElement(creator,'familyName')
    affiliation = ET.SubElement(creator,'affiliation')
    
    creator_name.text = '{}, {}'.format(names.get('last'), names.get('first'))
    given_name.text = names.get('first')
    family_name.text = names.get('last')
    affiliation.text = str(data[1])

    return creator

In [5]:
def extract_name(fullname: str) -> list:
    
    first_name, last_name = fullname.split(' ')
    names = {'first':first_name.strip(' '),'last':last_name.strip(' ')}
    
    return names

In [6]:
# Export file
def export_xml(xml: str) -> None:
    xml.write("creators.xml",encoding='utf-8', xml_declaration=True)

    '''
    with open('creators.xml','wb') as f:
        f.write(tostring(xml,encoding='UTF-8'))
    '''

In [7]:
#### Script ####

export_xml(build_creators_xml(extract_authors('test.csv')))

In [8]:
authors = extract_authors('test.csv')
authors

,author,affiliation
0,Yachao Chen,Department of Metallurgical and Materials Engi...
1,Geoff Brennecka,Department of Metallurgical and Materials Engi...
2,Vanessa Meschke,"Department of Materials Science & Engineering,..."
3,Kevin Cruse,"Department of Mechanical Engineering, Universi..."
4,Kelsey Cannon,Department of Metallurgical and Materials Engi...
5,Mary Dougherty,Department of Metallurgical and Materials Engi...
6,Katie Fisk,Department of Metallurgical and Materials Engi...


In [9]:
my_xml = build_creators_xml(authors)